In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# --- Dane ---
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[..., np.newaxis] / 255.0
x_test  = x_test[..., np.newaxis] / 255.0


In [ ]:

# --- Model CNN w Functional API ---
inputs = tf.keras.Input(shape=(28, 28, 1), name="input_image")

x = tf.keras.layers.Conv2D(16, (3,3), activation='relu', name="conv1")(inputs)
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu', name="conv2")(x)
x = tf.keras.layers.MaxPooling2D(name="pool1")(x)
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(64, activation='relu', name="dense1")(x)
outputs = tf.keras.layers.Dense(10, activation='softmax', name="predictions")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_cnn")

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, epochs=1, batch_size=128, validation_split=0.1)

In [ ]:
def compute_saliency_map(model, img, label=None):
    # Dodaj wymiar batcha
    img_tensor = tf.convert_to_tensor(img[np.newaxis, ...])

    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        preds = model(img_tensor)            # (1, 10)
        if label is None:
            label = tf.argmax(preds[0])
        loss = preds[0, label]

    # ∂loss / ∂img
    grads = tape.gradient(loss, img_tensor)  # (1, 28, 28, 1)

    # Bierzemy maks po kanale (tu i tak jest 1 kanał, ale niech będzie ogólnie)
    saliency = tf.reduce_max(tf.abs(grads), axis=-1)[0]  # (28, 28)

    # Normalizacja do [0,1] dla ładnej wizualizacji
    saliency -= tf.reduce_min(saliency)
    saliency /= (tf.reduce_max(saliency) + 1e-8)

    return saliency.numpy(), int(label.numpy())

In [ ]:
plt.figure(figsize=(10, 12))   # większa wysokość, bo 5 wierszy

for i in range(5):

    idx = i   # możesz zmienić: np. idx = np.random.randint(len(x_test))
    img = x_test[idx]
    true_label = y_test[idx]

    saliency, used_label = compute_saliency_map(model, img, label=None)

    # --- Oryginał ---
    plt.subplot(5, 3, i*3 + 1)
    plt.title(f"Oryginał ({true_label})")
    plt.imshow(img[:, :, 0], cmap='gray')
    plt.axis('off')

    # --- Saliency map ---
    plt.subplot(5, 3, i*3 + 2)
    plt.title("Saliency")
    plt.imshow(saliency, cmap='hot')
    plt.axis('off')

    # --- Overlay ---
    plt.subplot(5, 3, i*3 + 3)
    plt.title("Nałożenie")
    plt.imshow(img[:, :, 0], cmap='gray')
    plt.imshow(saliency, cmap='jet', alpha=0.4)
    plt.axis('off')

plt.tight_layout()
plt.show()